In [ ]:
# Change the dataset in global_vars.py to NAKO.
from global_vars import *
from commons import *

import glob 
import os

one_time_n4_optimization = True
vol_to_check_list = None
exclude = []

%load_ext autoreload
%autoreload 2

In [ ]:
data_dir

In [ ]:
def load_nako_file_paths(load_from_txt_file=True):
    volumes_to_use = []
    if load_from_txt_file:
        with open(volume_txt_file) as file_handle:
                volumes_to_use = file_handle.read().splitlines()
    else:
        volumes_to_use = [name for name in os.listdir(data_dir)]
    
    file_paths = {}
    
    for vol in volumes_to_use:
        if (vol_to_check_list is not None and vol not in vol_to_check_list) or (vol == "") or (vol in exclude):
            continue
        opp_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_opp/**.nii.gz') # **_2**
        in_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_in/**.nii.gz')
        f_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_F/**.nii.gz')
        w_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_W/**.nii.gz')
        
        labelmap_paths = glob.glob(f'{label_dir}/{vol}/**')
        
        vol_madals_paths = dict(
        OPP=opp_paths,
        IN=in_paths,
        F=f_paths,
        W=w_paths
        )
        file_paths[str(vol)]=dict(
            VOLUME_PATHS=vol_madals_paths,
            LABEL_PATHS=labelmap_paths,
        )
    return file_paths

file_paths = load_nako_file_paths()
file_paths

In [ ]:
# Individual RESCALING.
n4_dict = {}
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    print(f'n4 processing part-1 started with {vol}...')
    n4_dict[vol] = []
    vol_parts = [[file, read_ras(file)] for file in file_paths[vol]['VOLUME_PATHS']['IN']]
    for orig_file, in_image in vol_parts:
        n4_dict[vol].append(rescale(in_image, vol, orig_file))

In [ ]:
# SITK does ot work due to differences in pixel resolution of IN and corresponding OPP Scan.
# Only applying once at the end.
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    print(f'n4-biasfield-correction starting with {vol}...')
    for idx, n4_d in enumerate(n4_dict[vol]):
        in_file = n4_d['SCALED']
        opp_file = file_paths[vol]['VOLUME_PATHS']['OPP'][idx]
        new_filename = opp_file.split('/')[-1].split('.')[0]
        output_file = f'{n4_corrected_data_dir}/vol/{vol}/{new_filename}_n4_corrected_sitk.nii.gz'
        SITK_N4_normalization(in_file, opp_file, output_file)
        n4_dict[vol][idx]['OPP_CORRECTED'] = output_file

    file_paths[vol]['N4_1'] = n4_dict[vol]

In [ ]:
# STITCHING VOL PARTS HERE
for vol in file_paths.keys():
    try:
        print(f'started with {vol}...')
        if vol_to_check_list is not None and vol not in vol_to_check_list:
            continue
        create_if_not(f'{n4_corrected_data_dir}/vol/{vol}')
        file_paths[vol]['ONE'] = {}
        for modality_key in file_paths[vol]['VOLUME_PATHS'].keys():
            print(f"processing {modality_key}")
            orig_modal_key = modality_key
            if one_time_n4_optimization:
                vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]
            else:
                if modality_key == 'OPP':
                    vol_parts = [read_ras(data_dict['OPP_CORRECTED']) for data_dict in file_paths[vol]['N4_1']]
                    modality_key = modality_key+'_n4_corrected'
                else:
                    vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]

            ras_stitched = multi_vol_stitching(vol_parts)
            save_volume(ras_stitched, f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched', np.int16)
            file_paths[vol]['ONE'][f'{orig_modal_key}'] = f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched.nii.gz'
    except Exception as e:
        print('ERROR:',e)
        continue

In [ ]:
# RESCALING INTENSITIES OF STITCHED VOLUME ABOVE 0
n4_dict = {}
for vol in file_paths.keys():
    try:
        if vol_to_check_list is not None and vol not in vol_to_check_list:
            continue
        print(f'n4 processing part-2 started with {vol}...')
        n4_dict[vol] = {}
        in_stitched_file_path, in_stitched_img = file_paths[vol]['ONE']['IN'], read_ras(file_paths[vol]['ONE']['IN'])
        n4_dict[vol]['N4_2'] = rescale(in_stitched_img, vol, in_stitched_file_path)
        
    except Exception as e:
        print('ERROR:',e)
        continue

In [ ]:
all_n4_process = True
for vol in file_paths.keys():
    try:
        if vol_to_check_list is not None and vol not in vol_to_check_list:
            continue
        print(f'n4-biasfield-correction starting with {vol}...')
        in_file = n4_dict[vol]['N4_2']['SCALED']
        opp_file = file_paths[vol]['ONE']['OPP']
        if all_n4_process:
            w_file = file_paths[vol]['ONE']['W']
            w_of = w_file.split('/')[-1].split('.')[0]
            w_outputfile = f'{n4_corrected_data_dir}/vol/{vol}/{w_of}_n4_corrected_sitk.nii.gz'
            SITK_N4_normalization(in_file, w_file, w_outputfile)
            n4_dict[vol]['N4_2']['W_CORRECTED'] = w_outputfile
            
            f_file = file_paths[vol]['ONE']['F']
            f_of = f_file.split('/')[-1].split('.')[0]
            f_outputfile = f'{n4_corrected_data_dir}/vol/{vol}/{f_of}_n4_corrected_sitk.nii.gz'
            SITK_N4_normalization(in_file, f_file, f_outputfile)
            n4_dict[vol]['N4_2']['F_CORRECTED'] = f_outputfile
            
            inin_file = file_paths[vol]['ONE']['IN']
            in_of = inin_file.split('/')[-1].split('.')[0]
            in_outputfile = f'{n4_corrected_data_dir}/vol/{vol}/{in_of}_n4_corrected_sitk.nii.gz'
            SITK_N4_normalization(in_file, inin_file, in_outputfile)
            n4_dict[vol]['N4_2']['IN_CORRECTED'] = in_outputfile
        new_filename = opp_file.split('/')[-1].split('.')[0]
        output_file = f'{n4_corrected_data_dir}/vol/{vol}/{new_filename}_n4_corrected_sitk.nii.gz'
        SITK_N4_normalization(in_file, opp_file, output_file)
        n4_dict[vol]['N4_2']['OPP_CORRECTED'] = output_file
        file_paths[vol]['N4_2'] = n4_dict[vol]['N4_2']
    except Exception as e:
        print('ERROR:',e)
        continue

In [ ]:
def nako_vol_label_fix(vol, label):
    world_shape = np.max(np.array([list(vol.shape), list(label.shape)]), axis=0)
    final_label = np.zeros(tuple(world_shape))

    label_affine = label.affine
    vol_affine = vol.affine
    
    target_affine = vol_affine
    target_header = vol.header
    target_dim_v = vol.shape

    sx,sy,sz,ex,ey,ez = np.abs(get_points(label, vol))
    
    final_label[0:ex+sx, 0:ey+sy, sz:ez] = label.get_fdata()
    
    final_label = np.flip(final_label, axis=0)
    final_label = np.flip(final_label, axis=1)
    
    final_label_img = nb.Nifti1Image(final_label, target_affine, target_header)
    return vol, final_label_img

def nako_label_parts(label_parts, reference_labelmap=None):
    stitched_label = None
    mode = 'nearest'
    order = 0
    if reference_labelmap is None:
        label_shape = get_freequent_shape([img.shape for img, _, _ in label_parts])
        reference_labelmap = [img for img, _, _ in label_parts if list(img.shape) == list(label_shape)][0]
    else:
        label_shape = reference_labelmap.shape

    stitched_label = np.zeros(label_shape)
    for labelmap_img, lidx, lname in label_parts:
        print(lidx, lname)
        labelmap_img = makeit_3d(labelmap_img)
        labelmap_img = resample_from_to(labelmap_img, [label_shape, reference_labelmap.affine], order=order, mode=mode, cval=0)
        
        sx,sy,sz,ex,ey,ez = np.abs(get_points(labelmap_img, reference_labelmap))
        
        labelmap = labelmap_img.get_fdata()
        labelmap = np.multiply(lidx, labelmap)
        stitched_label[0:ex+sx, 0:ey+sy, 0:ez+sz] += labelmap
        
        print("###############################################################################################") 
        
    labelmap = np.round(stitched_label)
    stitched_labeled_img = nb.Nifti1Image(labelmap, reference_labelmap.affine, reference_labelmap.header)
    
    return stitched_labeled_img

In [ ]:
def nako_vol_label_allignment(file_paths):
    print("STARTING NAKO LABEL-MAPS.")
    print('Reading Label Maps.....')
    for vol in file_paths.keys():
        print(vol)
        if vol_to_check_list is not None and vol not in vol_to_check_list or (vol == "") or (vol in exclude):
            continue

        print(file_paths[vol]['LABEL_PATHS'])
        if len(file_paths[vol]['LABEL_PATHS']) == 0:
            print(f"#################### ALERT:: NO LABELPATHS IN THE DICTIONARY FOR {vol} #########################")
            continue
        volume = nb.load(file_paths[vol]['N4_2']['OPP_CORRECTED'])
        f_volume = nb.load(file_paths[vol]['N4_2']['F_CORRECTED'])
        w_volume = nb.load(file_paths[vol]['N4_2']['W_CORRECTED'])
        in_volume = nb.load(file_paths[vol]['N4_2']['IN_CORRECTED'])

        img_ras_list = []
        later = []
        for label_file_to_read in file_paths[vol]['LABEL_PATHS']:
            img_ras, lidx, labelname = read_ras(label_file_to_read, is_label=True)
            if labelname is None or img_ras is None:
                continue
            img_ras = makeit_3d(img_ras)

            if labelname in ['SPLEEN']:
                later.append([img_ras, lidx+LABEL_EXTENSION_FOR_OVERLAP_REMOVAL, labelname])
            else:
                img_ras_list.append([img_ras, lidx+LABEL_EXTENSION_FOR_OVERLAP_REMOVAL, labelname])

        img_ras_list.extend(later)

        s_label = nako_label_parts(img_ras_list)
        if SAMPLING:
            s_label = resample_to_output(s_label, TARGET_RESOLUTION, order=0, mode='nearest', cval=0)
        s_label = drop_overlapped_pixels(s_label, np.array(img_ras_list)[:, 1])
        volume, s_label = nako_vol_label_fix(volume, s_label)

        print('Viewing Stitched Images.....')
        volume_3_view_viewer(get_volume_data(volume))
        volume_3_view_viewer(get_volume_data(f_volume))
        volume_3_view_viewer(get_volume_data(w_volume))
        volume_3_view_viewer(get_volume_data(in_volume))
        volume_3_view_viewer(get_volume_data(s_label))

        print('Saving Processed & Stitched Image.....')
        save_volume(volume, f'{processed_dir}/volume/{vol}', np.int16)
        save_volume(f_volume, f'{processed_dir}/volume_f/{vol}', np.int16)
        save_volume(w_volume, f'{processed_dir}/volume_w/{vol}', np.int16)
        save_volume(in_volume, f'{processed_dir}/volume_in/{vol}', np.int16)
        save_volume(s_label, f'{processed_dir}/label/{vol}', np.int16)
        print('FINISHED.')

In [ ]:
nako_vol_label_allignment(file_paths)